###Import de libs necessarios

In [0]:
from pyspark.sql import functions as F
from unicodedata import normalize 
import pandas as pd
import re
from datetime import datetime

###Metodos utils

In [0]:
def transform_column_name(column_name):
  new_name = column_name.rstrip('\n').replace(' ', '_').lower()
  new_name = normalize('NFKD', new_name).encode('ASCII', 'ignore').decode('ASCII')
  new_name = re.sub(r"\W", "", new_name)
  return new_name
  
def read_excel(file, sheet_name):
  try:
    data_frame = pd.read_excel(f'file:/dbfs/mnt/devdh/inbound/scorecard/trabalho/{file}', sheet_name=sheet_name, engine = 'openpyxl')
  except Exception as e:
    print(e)
  data_frame = create_spark_dataFrame(data_frame, file)
  return data_frame 

def create_spark_dataFrame(data_frame, file):
  data_frame = data_frame.fillna(0)
  columns = pd.io.parsers.ParserBase({'names':data_frame.columns})._maybe_dedup_names(data_frame.columns.str.lower())
  column_names = list(map(transform_column_name, columns))
  data_frame = spark.createDataFrame(data_frame).toDF(*column_names)
  return data_frame

###Import Files

In [0]:
base_transacoes = read_excel('massa_de_dados_cliente.xlsx', 0)
base_contratos = read_excel('massa_de_dados_cliente.xlsx',1)

###Tratamentos e Tranformações

In [0]:
base_contratos = base_contratos.filter(F.col('is_active') == 'true').withColumnRenamed('client_id', 'client_id_contracts')

In [0]:
tabela_total = (base_contratos.join(base_transacoes, base_contratos.client_id_contracts == base_transacoes.client_id,'left')
                              .drop('client_id_contracts'))

###Calculos aplicados

In [0]:
tabela_total = tabela_total.withColumn('total_with_discount', F.lit(F.col('total_amount')-(F.col('total_amount')*F.col('discount_percentage') /100)))

In [0]:
tabela_total = tabela_total.withColumn('totala_acquirer_ltda', F.lit(F.col('percentage')* F.col('total_with_discount')/100 ))
display(tabela_total)

contract_id,client_name,percentage,is_active,transaction_id,client_id,total_amount,discount_percentage,total_with_discount,totala_acquirer_ltda
5,Lojas Italianas,1.0,true,3,3509,69998,0.0,69998.0,699.98
6,Carrerfive,3.0,true,4,3510,1,0.0,1.0,0.03
3,Magazine Luana,2.0,true,1,3545,3000,6.99,2790.3,55.806000000000004
3,Magazine Luana,2.0,true,2,3545,4500,0.45,4479.75,89.595


###Result

In [0]:
total = tabela_total.agg(F.sum('totala_acquirer_ltda').cast('float'))
print('The total is: R${:.2f}'.format(total.collect()[0][0]))

The total is: R$845.41